In [1]:
! curl -O https://www.antlr.org/download/antlr-4.13.1-complete.jar

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 2089k  100 2089k    0     0  21.8M      0 --:--:-- --:--:-- --:--:-- 22.4M


In [2]:
! java -jar antlr-4.13.1-complete.jar -Dlanguage=Python3 -visitor Quantum.g4

In [3]:
from antlr4 import *
from QuantumLexer import QuantumLexer
from QuantumParser import QuantumParser

class QuantumListener(ParseTreeListener):
    def enterQubitDecl(self, ctx):
        print(f"[DECLARE] qubit {ctx.ID().getText()}")

    def enterApplyGateStmt(self, ctx):
        gateNode = ctx.applyGate()
        gateName = gateNode.gate().getText()
        args = []
        if gateNode.argList():
            args = [a.getText() for a in gateNode.argList().arg()]
        print(f"[GATE] {gateName}({', '.join(args)})")

    def enterMeasureStmt(self, ctx):
        print(f"[MEASURE] {ctx.ID(0).getText()} -> {ctx.ID(1).getText()}")

    def enterGroverBlock(self, ctx):
        print(f"[GROVER] {ctx.ID().getText()} {{")
        
    def exitGroverBlock(self, ctx):
        print("}")


In [4]:
input_file = "example.q"
input_stream = FileStream(input_file, encoding='utf-8')

In [5]:
lexer = QuantumLexer(input_stream)
tokens = CommonTokenStream(lexer)
parser = QuantumParser(tokens)
tree = parser.prog()

In [6]:
print("== Árbol de análisis ==")
print(tree.toStringTree(recog=parser))

== Árbol de análisis ==
(prog (stmt (qubitDecl qubit q0 ;)) (stmt (qubitDecl qubit q1 ;)) (stmt (qubitDecl qubit aux ;)) (stmt (applyGateStmt (applyGate (gate h) ( (argList (arg q0)) )) ;)) (stmt (applyGateStmt (applyGate (gate h) ( (argList (arg q1)) )) ;)) (stmt (applyGateStmt (applyGate (gate x) ( (argList (arg q1)) )) ;)) (stmt (applyGateStmt (applyGate (gate cx) ( (argList (arg q1) , (arg aux)) )) ;)) (stmt (applyGateStmt (applyGate (gate x) ( (argList (arg q1)) )) ;)) (stmt (groverBlock grover search (block { (stmt (applyGateStmt (applyGate (gate u3) ( (argList (arg (expr 3.14)) , (arg (expr 0.5)) , (arg (expr 1.2))) ) ( (idList q0) )) ;)) (stmt (applyGateStmt (applyGate (gate rz) ( (argList (arg (expr 1.57))) ) ( (idList q1) )) ;)) (stmt (applyGateStmt (applyGate (gate cx) ( (argList (arg q0) , (arg q1)) )) ;)) }))) (stmt (measureStmt measure q0 -> c0 ;)) (stmt (measureStmt measure q1 -> c1 ;)) <EOF>)


In [7]:
walker = ParseTreeWalker()
listener = QuantumListener()
walker.walk(listener, tree)

[DECLARE] qubit q0
[DECLARE] qubit q1
[DECLARE] qubit aux
[GATE] h(q0)
[GATE] h(q1)
[GATE] x(q1)
[GATE] cx(q1, aux)
[GATE] x(q1)
[GROVER] search {
[GATE] u3(3.14, 0.5, 1.2)
[GATE] rz(1.57)
[GATE] cx(q0, q1)
}
[MEASURE] q0 -> c0
[MEASURE] q1 -> c1


In [8]:
from QuantumParser import QuantumParser
from QuantumVisitor import QuantumVisitor

class QuantumIRVisitor(QuantumVisitor):
    def __init__(self):
        self.instructions = []
        self.indent = 0
        
    def getIndent(self):
        return "  " * self.indent

    def visitQubitDecl(self, ctx):
        self.instructions.append(f"{self.getIndent()}DECLARE {ctx.ID().getText()}")
        return self.visitChildren(ctx)

    def visitApplyGateStmt(self, ctx):
        return self.visitApplyGate(ctx.applyGate())

    def visitApplyGate(self, ctx):
        gateName = ctx.gate().getText()
        args = []
        if ctx.argList():
            for arg in ctx.argList().arg():
                if arg.ID():
                    args.append(arg.ID().getText())
                elif arg.expr():
                    args.append(arg.expr().getText())
        self.instructions.append(f"{self.getIndent()}GATE {gateName}({', '.join(args)})")
        return self.visitChildren(ctx)

    def visitMeasureStmt(self, ctx):
        self.instructions.append(f"{self.getIndent()}MEASURE {ctx.ID(0).getText()} -> {ctx.ID(1).getText()}")
        return self.visitChildren(ctx)

    def visitGroverBlock(self, ctx):
        self.instructions.append(f"{self.getIndent()}GROVER {ctx.ID().getText()} {{")
        self.indent += 1
        result = self.visitBlock(ctx.block())
        self.indent -= 1
        self.instructions.append(f"{self.getIndent()}}}")
        return result

    def visitBlock(self, ctx):
        return self.visitChildren(ctx)

In [9]:
visitor = QuantumIRVisitor()
visitor.visit(tree)
for instr in visitor.instructions:
    print(instr)

DECLARE q0
DECLARE q1
DECLARE aux
GATE h(q0)
GATE h(q1)
GATE x(q1)
GATE cx(q1, aux)
GATE x(q1)
GROVER search {
  GATE u3(3.14, 0.5, 1.2)
  GATE rz(1.57)
  GATE cx(q0, q1)
}
MEASURE q0 -> c0
MEASURE q1 -> c1


In [10]:
from antlr4 import InputStream, CommonTokenStream
from antlr4.tree.Tree import TerminalNodeImpl
from QuantumLexer import QuantumLexer
from QuantumParser import QuantumParser

from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

# Archivo example.q
with open("example.q", "r", encoding="utf-8") as f:
    example_q = f.read()

# Parseamos el archivo example.q
input_stream = InputStream(example_q)
lexer = QuantumLexer(input_stream)
tokens = CommonTokenStream(lexer)
parser = QuantumParser(tokens)
tree = parser.prog()

# -------------------------
# Funciones auxiliares
# -------------------------
def _looks_like_number(s):
    try:
        float(s)
        return True
    except:
        return False

def parse_arg_text(arg_ctx):
    """Devuelve texto representativo de un arg (ID o expr)."""
    try:
        if hasattr(arg_ctx, "ID") and arg_ctx.ID():
            # arg ctx puede ser nodo ID
            idnode = arg_ctx.ID()
            # ID() podría devolver una lista o un solo nodo
            if isinstance(idnode, list):
                return idnode[0].getText()
            return idnode.getText()
    except Exception:
        pass
    try:
        if hasattr(arg_ctx, "expr") and arg_ctx.expr():
            return arg_ctx.expr().getText()
    except Exception:
        pass
    # respaldo
    try:
        return arg_ctx.getText()
    except:
        return ""

def proc_applyGateNode(applyGateCtx):
    """
    Extrae (gateName, params, targets) de un ApplyGateContext de forma robusta.
    """
    gateName = applyGateCtx.gate().getText()
    params = []
    targets = []

    # 1) intentar argList
    try:
        if hasattr(applyGateCtx, "argList") and applyGateCtx.argList():
            for a in applyGateCtx.argList().arg():
                params.append(parse_arg_text(a))
    except Exception:
        pass

    # 2) intentar idList() directamente si existe
    try:
        if hasattr(applyGateCtx, "idList") and applyGateCtx.idList():
            for idn in applyGateCtx.idList().ID():
                targets.append(idn.getText())
            return gateName, params, targets
    except Exception:
        pass

    # 3) respaldo: inspeccionar hijos buscando paréntesis con IDs
    children = list(applyGateCtx.getChildren())
    # Buscamos patrones "( ... ) ( id, id )" o similar
    # Recorremos y cuando encontremos un '(' tratamos de extraer tokens hasta su ')'
    for i, ch in enumerate(children):
        if isinstance(ch, TerminalNodeImpl) and ch.getText() == '(':
            # coleccionamos tokens hasta su ')'
            ids = []
            j = i + 1
            while j < len(children):
                nxt = children[j]
                txt = None
                if isinstance(nxt, TerminalNodeImpl):
                    txt = nxt.getText()
                    if txt == ')':
                        break
                    if txt == ',':
                        pass
                    else:
                        if txt.isidentifier():
                            ids.append(txt)
                else:
                    # es un nodo no-terminal; intentar obtener ID()
                    try:
                        if hasattr(nxt, "ID") and nxt.ID():
                            id_nodes = nxt.ID()
                            if isinstance(id_nodes, list):
                                for idn in id_nodes:
                                    ids.append(idn.getText())
                            else:
                                ids.append(id_nodes.getText())
                    except Exception:
                        # intentar getText()
                        try:
                            t = nxt.getText()
                            if t.isidentifier():
                                ids.append(t)
                        except:
                            pass
                j += 1
            if ids:
                # heurística: si hay identificadores, considerarlos targets
                non_numeric = [x for x in ids if not _looks_like_number(x)]
                if non_numeric:
                    targets = ids
                    break

    # 4) mover identificadores de params a targets si no hay targets
    if not targets:
        remaining_params = []
        for p in params:
            if isinstance(p, str) and p.isidentifier() and not _looks_like_number(p):
                targets.append(p)
            else:
                remaining_params.append(p)
        params = remaining_params

    return gateName, params, targets

# -------------------------
# Construir IR desde tree (uso de hasattr para evitar AttributeError)
# -------------------------
ir = []
declared_qubits = []
classical_names = []

for s in tree.stmt():
    # ¿qubitDecl?
    if hasattr(s, "qubitDecl") and s.qubitDecl():
        qname = s.qubitDecl().ID().getText()
        ir.append({'type':'declare_qubit','name':qname})
        declared_qubits.append(qname)
        continue

    # ¿applyGateStmt?
    if hasattr(s, "applyGateStmt") and s.applyGateStmt():
        applyCtx = s.applyGateStmt().applyGate()
        gname, params, targets = proc_applyGateNode(applyCtx)
        ir.append({'type':'gate','gate':gname,'params':params,'targets':targets})
        continue

    # ¿groverBlock?
    if hasattr(s, "groverBlock") and s.groverBlock():
        name = s.groverBlock().ID().getText()
        body = []
        for inner in s.groverBlock().block().stmt():
            # ¿applyGate interno?
            if hasattr(inner, "applyGateStmt") and inner.applyGateStmt():
                gctx = inner.applyGateStmt().applyGate()
                gname, params, targets = proc_applyGateNode(gctx)
                body.append({'type':'gate','gate':gname,'params':params,'targets':targets})
                continue
            if hasattr(inner, "measureStmt") and inner.measureStmt():
                src = inner.measureStmt().ID(0).getText()
                dst = inner.measureStmt().ID(1).getText()
                body.append({'type':'measure','src':src,'dst':dst})
                if dst not in classical_names:
                    classical_names.append(dst)
                continue
            if hasattr(inner, "qubitDecl") and inner.qubitDecl():
                qn = inner.qubitDecl().ID().getText()
                body.append({'type':'declare_qubit','name':qn})
                declared_qubits.append(qn)
                continue
            if hasattr(inner, "expr") and inner.expr():
                body.append({'type':'expr','txt': inner.expr().getText()})
                continue
        ir.append({'type':'grover','name':name,'body':body})
        continue

    # ¿measureStmt?
    if hasattr(s, "measureStmt") and s.measureStmt():
        src = s.measureStmt().ID(0).getText()
        dst = s.measureStmt().ID(1).getText()
        ir.append({'type':'measure','src':src,'dst':dst})
        if dst not in classical_names:
            classical_names.append(dst)
        continue

    # ¿expr?
    if hasattr(s, "expr") and s.expr():
        ir.append({'type':'expr','txt': s.expr().getText()})
        continue

    # respaldo: ignorar
    continue

# -------------------------
# Mapear qubits/clásicos e inicializar circuito
# -------------------------
qubit_map = {name: idx for idx, name in enumerate(declared_qubits)}
n_qubits = len(declared_qubits)
cl_map = {name: idx for idx, name in enumerate(classical_names)}
n_clbits = len(cl_map)

qc = QuantumCircuit(n_qubits, n_clbits)

def resolve_token(tok):
    try:
        return float(tok)
    except:
        if isinstance(tok, str) and tok in qubit_map:
            return qubit_map[tok]
        return tok

def apply_gate_entry(qc_obj, entry):
    g = entry['gate'].lower()
    params = [resolve_token(p) for p in entry.get('params',[])]
    targets = entry.get('targets',[])
    # si no hay targets, buscar en params identificadores
    if not targets:
        for p in entry.get('params',[]):
            if isinstance(p, str) and p in qubit_map:
                targets.append(p)
    # mapear targets simbólicos a índices
    target_indices = [qubit_map[t] for t in targets if t in qubit_map]

    if g == 'h' and target_indices:
        qc_obj.h(target_indices[0])
    elif g == 'x' and target_indices:
        qc_obj.x(target_indices[0])
    elif g == 'cx':
        if len(target_indices) >= 2:
            qc_obj.cx(target_indices[0], target_indices[1])
    elif g in ('u3','u'):
        if len(params) >= 3 and len(target_indices) >= 1:
            qc_obj.u(float(params[0]), float(params[1]), float(params[2]), target_indices[0])
    elif g == 'rz':
        if len(params) >= 1 and len(target_indices) >= 1:
            qc_obj.rz(float(params[0]), target_indices[0])
    elif g == 'cz':
        if len(target_indices) >= 2:
            qc_obj.cz(target_indices[0], target_indices[1])
    else:
        # respaldo: intentar llamada genérica si hay un solo target
        if target_indices:
            try:
                getattr(qc_obj, g)(target_indices[0])
            except Exception:
                pass

# -------------------------
# Reproducir IR y capturar estados alrededor de grover
# -------------------------
sim = AerSimulator()
state_before = None
state_after = None

for e in ir:
    if e['type'] == 'declare_qubit':
        continue
    if e['type'] == 'gate':
        apply_gate_entry(qc, e)
    if e['type'] == 'grover':
        # instantánea antes
        qc_before = qc.copy()
        qc_before.save_statevector()
        res_b = sim.run(transpile(qc_before, sim)).result()
        try:
            state_before = res_b.get_statevector()
        except Exception:
            state_before = None
        # aplicar ops internas
        for inner in e['body']:
            if inner['type'] == 'gate':
                apply_gate_entry(qc, inner)
        # instantánea después
        qc_after = qc.copy()
        qc_after.save_statevector()
        res_a = sim.run(transpile(qc_after, sim)).result()
        try:
            state_after = res_a.get_statevector()
        except Exception:
            state_after = None

# aplicar medidas
for e in ir:
    if e['type'] == 'measure':
        if e['src'] in qubit_map and e['dst'] in cl_map:
            qc.measure(qubit_map[e['src']], cl_map[e['dst']])

# si no hay bits clásicos pero sí medidas, reconstruir mapeo (heurística simple)
if len(cl_map) == 0:
    measures = [x for x in ir if x.get('type')=='measure']
    if measures:
        classical_names = []
        for m in measures:
            if m['dst'] not in classical_names:
                classical_names.append(m['dst'])
        cl_map = {name: idx for idx, name in enumerate(classical_names)}
        qc_new = QuantumCircuit(n_qubits, len(classical_names))
        # reaplicar ops (ingenuo)
        for e in ir:
            if e['type'] == 'gate':
                apply_gate_entry(qc_new, e)
            if e['type'] == 'grover':
                for inner in e['body']:
                    if inner['type']=='gate':
                        apply_gate_entry(qc_new, inner)
        for m in ir:
            if m['type']=='measure':
                qc_new.measure(qubit_map[m['src']], cl_map[m['dst']])
        qc = qc_new

# Simulación
res_final = sim.run(transpile(qc, sim), shots=1024).result()
counts = res_final.get_counts()

# Mostrar
print("=== Circuito Construido ===")
print(qc.draw(output='text'))

print("\n=== Estado antes de Grover ===")
print(state_before)

print("\n=== Estado después de Grover ===")
print(state_after)

print("\n=== Resultados de medición ===")
print(counts)

plot_histogram(counts)
plt.show()


=== Circuito Construido ===
     ┌───┐┌─────────────────┐                           ┌─┐   
q_0: ┤ H ├┤ U(3.14,0.5,1.2) ├────────────────────────■──┤M├───
     ├───┤└──────┬───┬──────┘     ┌───┐┌──────────┐┌─┴─┐└╥┘┌─┐
q_1: ┤ H ├───────┤ X ├─────────■──┤ X ├┤ Rz(1.57) ├┤ X ├─╫─┤M├
     └───┘       └───┘       ┌─┴─┐└───┘└──────────┘└───┘ ║ └╥┘
q_2: ────────────────────────┤ X ├───────────────────────╫──╫─
                             └───┘                       ║  ║ 
c: 2/════════════════════════════════════════════════════╩══╩═
                                                         0  1 

=== Estado antes de Grover ===
Statevector([0. +0.j, 0. +0.j, 0.5+0.j, 0.5+0.j, 0.5+0.j, 0.5+0.j, 0. +0.j,
             0. +0.j],
            dims=(2, 2, 2))

=== Estado después de Grover ===
Statevector([ 1.10696226e-17+2.85354580e-17j,
              1.40645363e-01+4.79961623e-01j,
              2.01512128e-01-4.57436974e-01j,
             -2.68650671e-17-1.47023464e-17j,
             -4.57276359e-01